Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.8 MB/s eta 0:00:00


Cirq / NumPy — genel ve verilen 3×3 örneği

In [2]:
# cirq_eig_example.py  (uses numpy; Cirq only for unitary-check examples if needed)
import numpy as np

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm != 0 else v

# --- Genel 2x2 örnek (a b; c d) için eigen decomposition ---
A2 = np.array([[1, 2],
               [3, 4]], dtype=complex)
eigvals2, eigvecs2 = np.linalg.eig(A2)
print("A2 eigenvalues:", eigvals2)
print("A2 eigenvectors (columns):\n", eigvecs2)

# Normalize eigenvectors and verify A v = lambda v
for i, lam in enumerate(eigvals2):
    v = eigvecs2[:, i]
    v_norm = normalize(v)
    lhs = A2 @ v_norm
    rhs = lam * v_norm
    print(f"\n2x2 eigenpair {i}: lambda={lam}")
    print("||lhs - rhs|| =", np.linalg.norm(lhs - rhs))  # should be ~0

# --- Given 3x3 matrix from your example ---
A = np.array([[1, 0, 4],
              [0, 1, 2j],
              [4, -2j, 0]], dtype=complex)

eigvals, eigvecs = np.linalg.eig(A)
print("\nGiven 3x3 A eigenvalues:", eigvals)   # expect [1,5,-4] up to ordering
print("Eigenvectors (columns):\n", eigvecs)

# Normalize eigenvectors and check
for i, lam in enumerate(eigvals):
    v = eigvecs[:, i]
    v_norm = normalize(v)
    print(f"\nEigenvalue {i}: lambda = {lam}")
    print("Normalized eigenvector:", v_norm)
    # verify A v = lambda v
    err = np.linalg.norm(A @ v_norm - lam * v_norm)
    print("Verification error ||A v - lambda v|| =", err)

# Hermitian check
is_herm = np.allclose(A.conj().T, A)
print("\nA is Hermitian?", is_herm)
if is_herm:
    print("Eigenvalues (should be real):", np.real_if_close(eigvals))

# Example: unitary operator eigenvalue property
U = (1/np.sqrt(2)) * np.array([[1, 1], [-1, 1]], dtype=complex)  # example unitary
print("\nU eigenvalues (unit modulus expected):", np.linalg.eigvals(U))
print("moduli:", np.abs(np.linalg.eigvals(U)))


A2 eigenvalues: [-0.37228132+0.j  5.37228132+0.j]
A2 eigenvectors (columns):
 [[ 0.82456484+0.j  0.41597356+0.j]
 [-0.56576746-0.j  0.90937671+0.j]]

2x2 eigenpair 0: lambda=(-0.3722813232690143+0j)
||lhs - rhs|| = 5.551115123125783e-17

2x2 eigenpair 1: lambda=(5.372281323269014+0j)
||lhs - rhs|| = 4.440892098500626e-16

Given 3x3 A eigenvalues: [-4.+0.j  5.+0.j  1.+0.j]
Eigenvectors (columns):
 [[-0.59628479+0.00000000e+00j  0.66666667+0.00000000e+00j
   0.        +4.47213595e-01j]
 [ 0.        -2.98142397e-01j -0.        +3.33333333e-01j
   0.89442719+0.00000000e+00j]
 [ 0.74535599+0.00000000e+00j  0.66666667-0.00000000e+00j
   0.        -7.45604291e-17j]]

Eigenvalue 0: lambda = (-4+0j)
Normalized eigenvector: [-0.59628479+0.j         0.        -0.2981424j  0.74535599+0.j       ]
Verification error ||A v - lambda v|| = 1.6616296724220897e-15

Eigenvalue 1: lambda = (5.000000000000004+0j)
Normalized eigenvector: [0.66666667+0.j         0.        +0.33333333j 0.66666667-0.j        ]


Qiskit — Operator + NumPy (eigen decomposition NumPy ile)

In [3]:
# qiskit_eig_example.py
import numpy as np
from qiskit.quantum_info import Operator

# Use same A as above
A = np.array([[1, 0, 4],
              [0, 1, 2j],
              [4, -2j, 0]], dtype=complex)

# Wrap as Qiskit Operator (for convenience/printing)
# Note: Operator currently expects 2^n dims; but we can still use it for small matrices if shape okay.
# We'll use numpy.linalg.eig for decomposition
eigvals, eigvecs = np.linalg.eig(A)
print("Eigenvalues (Qiskit/Numpy):", eigvals)

# Normalize eigenvectors and verify using Qiskit Operator multiplication (if square)
op = Operator(A) if A.shape[0] in (1,2,4,8) else None
# verify A v = lambda v using numpy
for i, lam in enumerate(eigvals):
    v = eigvecs[:, i]
    v_norm = v / np.linalg.norm(v)
    print(f"\nλ = {lam}")
    print("||A v_norm - λ v_norm|| =", np.linalg.norm(A @ v_norm - lam * v_norm))

# Hermitian test
is_herm = np.allclose(A.conj().T, A)
print("\nIs A Hermitian?", is_herm)
if is_herm:
    print("Eigenvalues are real (approx):", np.real_if_close(eigvals))


Eigenvalues (Qiskit/Numpy): [-4.+0.j  5.+0.j  1.+0.j]

λ = (-4+0j)
||A v_norm - λ v_norm|| = 1.6616296724220897e-15

λ = (5.000000000000004+0j)
||A v_norm - λ v_norm|| = 4.5288390936029406e-15

λ = (1+0j)
||A v_norm - λ v_norm|| = 5.986322723036164e-16

Is A Hermitian? True
Eigenvalues are real (approx): [-4.  5.  1.]


QuTiP — doğrudan Qobj.eigenstates() kullanımı

In [5]:
# qutip_eig_example.py
import numpy as np
import qutip as qt

A_q = qt.Qobj([[1, 0, 4],
               [0, 1, 2j],
               [4, -2j, 0]], dims=[[3],[3]])  # Qobj

# eigenstates returns (eigvals, eigkets)
eigvals, eigstates = A_q.eigenstates()
print("QuTiP eigenvalues:", eigvals)

# eigstates is list of Qobj kets; normalize is automatic in eigenstates()
for i, lam in enumerate(eigvals):
    ket = eigstates[i]
    # check normalization
    norm = (ket.dag() * ket)[0,0]
    # verify A|v> = lambda |v>
    lhs = A_q * ket
    rhs = lam * ket
    diff_norm = (lhs - rhs).norm()
    print(f"\nEigen {i}: λ={lam}, ||A|v> - λ|v>|| = {diff_norm}, norm(|v|) = {norm}")

# Hermitian check
print("\nA is Hermitian?:", A_q.isherm)
if A_q.isherm:
    print("Eigenvalues are real (QuTiP):", eigvals)


QuTiP eigenvalues: [-4.  1.  5.]


TypeError: 'complex' object is not subscriptable